# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [32]:
#%%bash 
#%pip install duckdb
#%pip install jupysql
#%pip install duckdb-engine


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

There's a new jupysql version available (0.10.1), you're running 0.10.0. To upgrade: pip install jupysql --upgrade


Y ya podemos usar **SQL** dentro de una notebook!

In [2]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [3]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('C:\Users\feder\Documents\Maestria_en_Ciencia_de_datos\4_DM_en_Economia_y_Finanzas\datasets/competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [4]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [5]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [6]:
%%sql 
with mes_1 as (
select numero_de_cliente
from competencia_01
where foto_mes = '202105')

select comp.numero_de_cliente,
comp.foto_mes,
c.numero_de_cliente
from competencia_01 comp
left join mes_1 c
    on comp.numero_de_cliente = c.numero_de_cliente
where c.numero_de_cliente is null

,numero_de_cliente,foto_mes,numero_de_cliente_2
0,29282120,202103,NaN
1,29358824,202103,NaN
2,29379652,202103,NaN
3,29413956,202103,NaN
4,29479114,202103,NaN
...,...,...,...
2976,181504495,202104,NaN
2977,182321662,202104,NaN
2978,182463626,202104,NaN
2979,185451295,202104,NaN


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [5]:
%%sql 
select numero_de_cliente,
foto_mes
from competencia_01
where numero_de_cliente = 29199353 --in (29282120, 181504495, 29183981) -- primer caso baja +1, segundo caso baja +2
order by 2


,numero_de_cliente,foto_mes
0,29199353,202105


## Query final para obtener competencia_01

In [23]:
%%sql

create or replace temp table competencia_01_ternaria as 
    with clientes as (
        select 
            numero_de_cliente,
            foto_mes,
            last_day(strptime(foto_mes, '%Y%m') + INTERVAL 1 MONTH) as mes_mas_1,
            last_day(strptime(foto_mes, '%Y%m') + INTERVAL 2 MONTH) as mes_mas_2,
        from competencia_01
    ),

    mes_1 as (
        select 
            c.numero_de_cliente,
            c.foto_mes,
            c1.numero_de_cliente as cliente_mas_1
        from clientes c 
        left join competencia_01 c1
            on c.numero_de_cliente = c1.numero_de_cliente
            and c.mes_mas_1 = last_day(strptime(c1.foto_mes, '%Y%m'))
     ),

    mes_2 as (
        select 
            c.numero_de_cliente,
            c.foto_mes,
            c1.numero_de_cliente as cliente_mas_2
        from clientes c 
        left join competencia_01 c1
            on c.numero_de_cliente = c1.numero_de_cliente
            and c.mes_mas_2 = last_day(strptime(c1.foto_mes, '%Y%m'))
    ),

    max_mes as (
        select 
            strptime(MAX(foto_mes),'%Y%m') as max_foto
        from competencia_01
        --group by 1
    ),

    final as (
    select 
        c1.*,      
        case
            when 
                cliente_mas_1 is not null 
                and cliente_mas_2 is null
                and strptime(c1.foto_mes, '%Y%m') = m.max_foto - INTERVAL 2 MONTH
            then 'BAJA+2'
            when 
                cliente_mas_1 is null 
                and cliente_mas_2 is null
                and strptime(c1.foto_mes, '%Y%m') <= m.max_foto - INTERVAL 1 MONTH
            then 'BAJA+1'
            when 
                (strptime(c1.foto_mes, '%Y%m') <= m.max_foto - INTERVAL 1 MONTH and cliente_mas_1 is not null)
                or 
                cliente_mas_2 is not null 
            then 'CONTINUA'
            else null end clase_ternaria
    from competencia_01 c1
    left join mes_1 m1 
        on m1.cliente_mas_1 = c1.numero_de_cliente
        and c1.foto_mes = m1.foto_mes
    left join mes_2 m2 
        on m2.cliente_mas_2 = c1.numero_de_cliente
        and c1.foto_mes = m2.foto_mes
    cross join max_mes m
    )
    
    select 
       *
    from final 


,Success


In [26]:
%%sql
select clase_ternaria, count(*) 
from competencia_01_ternaria
where foto_mes = '202105'
group by 1

,clase_ternaria,count_star()
0,NaN,164102


In [17]:
%%sql
-- COPY competencia_01_ternaria TO 'C:\Users\feder\Documents\Maestria_en_Ciencia_de_datos\4_DM_en_Economia_y_Finanzas\datasets/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


## Pruebas con LAG

In [52]:
%%sql
select 
numero_de_cliente,
foto_mes,
LAG(foto_mes) OVER(partition by numero_de_cliente order by foto_mes desc) - foto_mes as dif_1,
LAG(foto_mes,2) OVER(partition by numero_de_cliente order by foto_mes desc)- foto_mes as dif_2 
from competencia_01


,numero_de_cliente,foto_mes,dif_1,dif_2
0,29185646,202105,NaN,NaN
1,29185646,202104,1.0,NaN
2,29185646,202103,1.0,2.0
3,29188883,202105,NaN,NaN
4,29188883,202104,1.0,NaN
...,...,...,...,...
491058,186316072,202105,NaN,NaN
491059,186316072,202104,1.0,NaN
491060,186332374,202105,NaN,NaN
491061,186332374,202104,1.0,NaN


In [104]:
%%sql

with clientes as (
select 
*,
LAG(foto_mes) 
    OVER(partition by numero_de_cliente order by foto_mes desc) as mes_mas_1,
LAG(foto_mes,2) 
    OVER(partition by numero_de_cliente order by foto_mes desc) as mes_mas_2,
LEAD(foto_mes) 
    OVER(partition by numero_de_cliente order by foto_mes desc) as mes_menos_1,
LEAD(foto_mes,2) 
    OVER(partition by numero_de_cliente order by foto_mes desc) as mes_menos_2
from competencia_01),

final as (
select
    numero_de_cliente,
    foto_mes,
    mes_mas_1,
    mes_mas_2,
    mes_menos_1,
    mes_menos_2,
    case 
        when ((mes_mas_1 = foto_mes+2) or (mes_mas_2 = foto_mes+2))  is not null then 'CONTINUA'
        when ((mes_mas_1 is null) 
            and (mes_mas_2 is null) 
            and (mes_menos_1 is not null) 
            and (mes_menos_2 is not null))
            then null
        when foto_mes= (select max(foto_mes) from clientes) then null
        when ((mes_mas_1 = foto_mes+1) and (mes_menos_1 = foto_mes-1)) or (mes_menos_1 is not null) then 'CONTINUA'
        when foto_mes<> (select max(foto_mes) from clientes) and mes_mas_1 is null and mes_mas_2 is null then 'BAJA+1'
        when foto_mes<> (select max(foto_mes) from clientes) and ((mes_mas_1 = foto_mes+1) and (mes_mas_2 is null)) then 'BAJA+2'
        end as clase_ternaria
from clientes
where 1=1
    and foto_mes = '202103'
    --and dif_1 = 1
)

select 
clase_ternaria, count(numero_de_cliente)
from final
where 1=1
--and clase_ternaria = 'CONTINUA'
group by 1



/*
select numero_de_cliente,foto_mes,mes_mas_1, mes_mas_2,
    case 
        when mes_mas_2 is null then 'BAJA+2'
        when mes_mas_1 is null and mes_mas_2 is null then 'BAJA+1'
        when ((mes_mas_1 = foto_mes+2) or (mes_mas_2 = foto_mes+2))  is not null then 'CONTINUA'
        end as clase_ternaria
    from clientes*/

,clase_ternaria,count(numero_de_cliente)
0,CONTINUA,161312
1,BAJA+2,963
2,BAJA+1,1049


In [21]:
%%sql
select 
clase_ternaria, count(numero_de_cliente)
from final
where 1=1
and fecha_mes = 202103
--and clase_ternaria = 'CONTINUA'
group by 1


RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.CatalogException) Catalog Error: Table with name final does not exist!
Did you mean "information_schema.tables"?
LINE 3: from final
             ^
[SQL: select 
clase_ternaria, count(numero_de_cliente)
from final
where 1=1
and fecha_mes = 202103
--and clase_ternaria = 'CONTINUA'
group by 1]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


## Feature Engineering

In [14]:
%%sql
select
    Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
from competencia_01

/*Tengo varios campos y quiero quedarme con el mayor y el menor valor de estos.*/

,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor
0,-639.0,-639.0,-639.0,-639.0
1,-670.0,-1035.0,-670.0,-1035.0
2,NaN,-2039.0,-2039.0,-2039.0
3,-578.0,-1217.0,-578.0,-1217.0
4,-455.0,-578.0,-455.0,-578.0
...,...,...,...,...
491058,-2190.0,-2190.0,-2190.0,-2190.0
491059,-2190.0,-2190.0,-2190.0,-2190.0
491060,-2190.0,-2190.0,-2190.0,-2190.0
491061,-2190.0,-2190.0,-2190.0,-2190.0


In [15]:
%%sql

select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01

--operar dos variables, como por ejemplo sumarla.

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,101627.07,229958.68,331585.75
1,56055.25,44453.56,100508.81
2,NaN,3941.32,NaN
3,0.00,35412.09,35412.09
4,0.00,324480.94,324480.94
...,...,...,...
491058,0.00,0.00,0.00
491059,0.00,0.00,0.00
491060,0.00,0.00,0.00
491061,0.00,0.00,0.00


In [16]:
%%sql

select
    Master_msaldototal
    , Visa_msaldototal
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01

--Pero un DS de a de veras mirará los datos y se encontrará con un campo que es null cuando se lo suma a otro dará null. Esto no siempre es deseable y puede ser fácilmente evitable

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,101627.07,229958.68,331585.75
1,56055.25,44453.56,100508.81
2,NaN,3941.32,3941.32
3,0.00,35412.09,35412.09
4,0.00,324480.94,324480.94
...,...,...,...
491058,0.00,0.00,0.00
491059,0.00,0.00,0.00
491060,0.00,0.00,0.00
491061,0.00,0.00,0.00


In [17]:
%%sql

CREATE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

select distinct
    Master_msaldototal
    , Visa_msaldototal
    , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01


,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,32819.44,22520.20,55339.64
1,0.00,0.00,0.00
2,0.00,39475.87,39475.87
3,NaN,10482.48,10482.48
4,31840.99,40341.30,72182.29
...,...,...,...
414899,NaN,8887.78,8887.78
414900,0.00,9219.43,9219.43
414901,0.00,24058.20,24058.20
414902,0.00,32556.17,32556.17


In [19]:
%%sql

select
    foto_mes
    , cliente_antiguedad
    , row_number() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad desc) as cliente_antiguedad_7
from competencia_01
order by cliente_antiguedad


,foto_mes,cliente_antiguedad,cliente_antiguedad_2,cliente_antiguedad_3,cliente_antiguedad_4,cliente_antiguedad_5,cliente_antiguedad_6,cliente_antiguedad_7
0,202104,1,1,0.000000,0.001289,1,1,10
1,202104,1,2,0.000000,0.001289,1,1,10
2,202104,1,3,0.000000,0.001289,1,1,10
3,202104,1,4,0.000000,0.001289,1,1,10
4,202104,1,5,0.000000,0.001289,1,1,10
...,...,...,...,...,...,...,...,...
491058,202105,377,164100,0.992041,0.999994,4,10,1
491059,202105,377,164101,0.992041,0.999994,4,10,1
491060,202103,481,163324,1.000000,1.000000,4,10,1
491061,202104,482,163637,1.000000,1.000000,4,10,1


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [ ]:
%%sql
-- introduzca sus queries

Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [ ]:
%%sql
COPY competencia_01 TO '/home/aleb/dmeyf23/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo